In [16]:
import pandas as pd
import pickle
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import clear_output

In [19]:
def getpage(url):
    page = requests.get(url)
    return BeautifulSoup(page.text, 'html.parser')

class stack:
    def __init__(self):
        self.__index = []

    def __len__(self):
        return len(self.__index)

    def push(self,item):
        self.__index.insert(0,item)

    def peek(self):
        if len(self) == 0:
            raise Exception("peek() called on empty stack.")
        return self.__index[0]

    def pop(self):
        if len(self) == 0:
            raise Exception("pop() called on empty stack.")
        return self.__index.pop(0)

    def __str__(self):
        return str(self.__index)

In [57]:
database = pd.read_pickle('databasefromhackernoon.pickle')
headings = pd.read_pickle('headingsfromhackernoon.pickle')
paragraphs = pd.read_pickle('paragraphsfromhackernoon.pickle')

In [58]:
scrapedurls = {}

In [39]:
chrome_options = Options()
chrome_options.add_argument('--headless')
links = []
for pagenumber in range(97):
    clear_output(wait=True)
    pagenumber = pagenumber + 1
    print(pagenumber)
    url = f'https://101blockchains.com/blog/page/{pagenumber}/'
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)
    time.sleep(3)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    headings_ = soup.find_all('h2')
    for heading_ in headings_:
        links.append(heading_.a['href'])
        print(heading_.a['href'])
    driver.close()

97
https://101blockchains.com/whats-all-the-fuss-about-blockchain/
https://101blockchains.com/why-blockchain-technology-is-going-to-be-massive/
https://101blockchains.com/monero-xmr/


In [40]:
len(links)

963

In [59]:
i = 0
headingdict = {'h1':1, 'h2':2, 'h3':3, 'h4':4, 'h5':5, 'h6':6}
for url in links:
    if url in scrapedurls:
        continue
    clear_output(wait=True)
    topic = False
    lastheadingid = 0
    lastheadingnumber = 0
    lastparagraphid = 0
    totalsubheadings = 0
    totalparagraphs = 0
    insideheading = stack()
    try:    
        soup = getpage(url)
        main = soup.article
        scrapedurls[url] = True
        descendants = main.descendants
    except:
        continue

    for element in descendants:
        try:
            if topic==False:
                if str(element.name)=='h1':
                    docid = len(database.index)
                    topicid = len(headings.index)
#                         database.loc[len(database.index)] = [topicid, url]
                    headings.loc[topicid] = ['topic', element.text, None, None, []]
                    topic = True
                    lastheadingid = topicid
                    lastheadingnumber = 0
                    continue
                else:
                    continue

            if element.name in headingdict:
                if element.parent.name=='div':
                    totalsubheadings+=1

                    headingid = len(headings.index)

                    headingnumber = headingdict[str(element.name)]
                    headings.loc[headingid] = [element.name, element.text, None, None, []]

                    if lastheadingnumber==0:
                        insideheading.push([lastheadingnumber, lastheadingid])
                        headings.iloc[lastheadingid]['subheading'] = headingid
                    elif headingnumber>lastheadingnumber:
                        insideheading.push([lastheadingnumber, lastheadingid])
                        headings.iloc[lastheadingid]['subheading'] = headingid
                    elif headingnumber==lastheadingnumber:
                        headings.iloc[lastheadingid]['nextheading'] = headingid
                    else:
                        while True:
                            try:
                                poppedheading = insideheading.pop()
                                if poppedheading[0]>headingnumber:
                                    continue
                                elif poppedheading[0]==headingnumber:
                                    headings.iloc[poppedheading[1]]['nextheading'] = headingid
                                    break
                                else:
                                    insideheading = stack()
                                    headings.iloc[topicid]['subheading'] = headingid
                                    break
                            except:
                                insideheading = stack()
                                break
                    lastheadingid = headingid
                    lastheadingnumber = headingnumber
            elif element.name=='p' and element.parent.name=='div':

                if len(element.text)==0:
                    continue
                totalparagraphs+=1
                paragraphid = len(paragraphs.index)
                paragraphs.loc[paragraphid] = ['p', element.text]
                headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                lastparagraphid = paragraphid
            elif element.name=='ol' and element.parent.name=='div':

                totalparagraphs+=1

                for li in element.find_all('li'):
                    paragraphid = len(paragraphs.index)
                    paragraphs.loc[paragraphid] = ['oli', li.text]
                    headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                    lastparagraphid = paragraphid
            elif element.name=='ul' and element.parent.name=='div':

                totalparagraphs+=1

                for li in element.find_all('li'):
                    paragraphid = len(paragraphs.index)
                    paragraphs.loc[paragraphid] = ['uli', li.text]
                    headings.iloc[lastheadingid]['paragraphs'].append(paragraphid)
                    lastparagraphid = paragraphid
            elif element.name=='footer':
                break

        except Exception as e:
            print(element, e)
            break

    database.loc[docid] = [topicid, url, totalsubheadings, totalparagraphs]
    print(docid, url)
    if i%25==0 and i!=0:
        j = 0
        for databaseid in database.index:
            if database.iloc[databaseid]['totalsubheadings']+database.iloc[databaseid]['totalparagraphs']!=0:
                j+=1
        print(j)
        print('sleeping...')
        time.sleep(10)
        i+=1
    else:
        i+=1

8816 https://101blockchains.com/monero-xmr/


In [60]:
database

,topicheadingID,url,totalsubheadings,totalparagraphs
0,0,https://hackernoon.com/is-blockchain-based-wat...,6,13
1,7,https://hackernoon.com/pros-and-cons-of-develo...,5,26
2,13,https://hackernoon.com/what-is-stabila-coin-an...,1,30
3,15,https://hackernoon.com/doodles-a-brand-exceedi...,5,20
4,21,https://hackernoon.com/self-reliance-is-the-on...,1,12
...,...,...,...,...
8812,64650,https://101blockchains.com/xlm-vs-xrp-which-is...,6,12
8813,64657,https://101blockchains.com/blockchain-is-a-uni...,3,12
8814,64661,https://101blockchains.com/whats-all-the-fuss-...,4,16
8815,64666,https://101blockchains.com/why-blockchain-tech...,6,18


In [61]:
headings

,tag,headingtext,subheading,nextheading,paragraphs
0,topic,Is Blockchain-Based Watch-to-Earn Monetization...,2,None,[]
1,h3,@ishantechIshanOnTech,None,None,[0]
2,h2,What Is Happening?,None,3,[1]
3,h2,What is Coub.com?,None,4,"[2, 3]"
4,h2,How is Coubâs Web3 Video Space Going to Work?,None,5,"[4, 5, 6, 7]"
...,...,...,...,...,...
64678,h3,Pros and Cons of Monero Cryptocurrency,None,64679,"[308864, 308865, 308866, 308867, 308868, 30886..."
64679,h3,How to Buy Monero Cryptocurrency?,None,64680,"[308876, 308877, 308878]"
64680,h3,How to Mine Monero Cryptocurrency?,None,64681,"[308879, 308880, 308881]"
64681,h3,How Monero Cryptocurrency Improves the Privacy?,None,64682,"[308882, 308883, 308884]"


In [62]:
paragraphs

,tag,text
0,p,"Covering the latest events, insights and views..."
1,p,"Coub.com, a micro-video streaming social netwo..."
2,p,Coub is a video-sharing website that is access...
3,p,"On Coub, users can take any video, trim the be..."
4,p,In order to preserve the intrinsic fluidity of...
...,...,...
308885,p,Cryptocurrencies are increasingly becoming a b...
308886,p,You can easily trade Monero on popular cryptoc...
308887,p,The popularity of Monero is yet to increase in...
308888,p,Join our annual/monthly membership program and...


In [63]:
database.to_pickle('databasefromhnandbc101.pickle')
headings.to_pickle('headingsfromhnandbc101.pickle')
paragraphs.to_pickle('paragraphsfromhnandbc101.pickle')